In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# load data
df = pd.read_csv('../data/cardio_train_miss.csv', delimiter=",")
df.dtypes
df.head()

,Unnamed: 0,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,0,18393,2,168.0,62.0,110,80,1,1,0,0,1,0
1,1,1,20228,1,156.0,85.0,140,90,3,1,0,0,1,1
2,2,2,18857,1,165.0,NaN,130,70,3,1,0,0,0,1
3,3,3,17623,2,169.0,82.0,150,100,1,1,0,0,1,1
4,4,4,17474,1,156.0,56.0,100,60,1,1,0,0,0,0


In [2]:
# lấy y
y = df.cardio
# lấy x
X = df.drop(['cardio'], axis=1)
X = X.iloc[:, 2:]
X = X.select_dtypes(exclude=['object'])
X.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,18393,2,168.0,62.0,110,80,1,1,0,0,1
1,20228,1,156.0,85.0,140,90,3,1,0,0,1
2,18857,1,165.0,NaN,130,70,3,1,0,0,0
3,17623,2,169.0,82.0,150,100,1,1,0,0,1
4,17474,1,156.0,56.0,100,60,1,1,0,0,0


In [50]:
# X.loc[X['BloodPressure'] == 66, 'BloodPressure'] = np.nan
# X.loc[X['SkinThickness'] == 35, 'SkinThickness'] = np.nan
# X.to_csv('../data/diabetes_miss.csv')

# X.loc[X['height'] == 160, 'height'] = np.nan
# X.loc[X['weight'] == 64.0, 'weight'] = np.nan
# X.to_csv('../data/cardio_train_miss.csv')

In [3]:
print('Number of features:' , len(X.columns))
print('Number of instance:', len(X.index))

Number of features: 11
Number of instance: 70000


In [4]:
X.isnull().sum()

age               0
gender            0
height         5022
weight         1592
ap_hi             0
ap_lo             0
cholesterol       0
gluc              0
smoke             0
alco              0
active            0
dtype: int64

In [5]:
X

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,18393,2,168.0,62.0,110,80,1,1,0,0,1
1,20228,1,156.0,85.0,140,90,3,1,0,0,1
2,18857,1,165.0,NaN,130,70,3,1,0,0,0
3,17623,2,169.0,82.0,150,100,1,1,0,0,1
4,17474,1,156.0,56.0,100,60,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
69995,19240,2,168.0,76.0,120,80,1,1,1,0,1
69996,22601,1,158.0,126.0,140,90,2,2,0,0,1
69997,19066,2,183.0,105.0,180,90,3,1,0,1,0
69998,22431,1,163.0,72.0,135,80,1,2,0,0,0


In [17]:
def processing(columnId, X): # Hàm xử lý để chuẩn hoá giá trị của cột có id là columnId về mean = 0 và std = 1
    mean = X[:, columnId].mean()
    std = X[:, columnId].std()
    X[:, columnId] = (X[:, columnId] - mean) / std
    return X

In [20]:
# Thay thế các giá trị bị miss bằng trung bình giá trị trong cột đó
from sklearn.impute import SimpleImputer

my_impute = SimpleImputer(strategy='mean')
imputed_X = pd.DataFrame(my_impute.fit_transform(X))

# Imputation removed column names; put them back
imputed_X.columns = X.columns
imputed_X = np.array(imputed_X)
# processing 
imputed_X = processing(0, imputed_X)
imputed_X = processing(2, imputed_X)
imputed_X = processing(3, imputed_X)
imputed_X = processing(4, imputed_X)
imputed_X = processing(5, imputed_X)

imputed_X = pd.DataFrame(imputed_X)
# Lưu data và label sử dụng pickle
import pickle
with open('cardio/X_mean.pkl', 'wb') as f:
    pickle.dump(imputed_X, f)
with open('cardio/y.pkl', 'wb') as f:
    pickle.dump(y, f)
    
imputed_X


,0,1,2,3,4,5,6,7,8,9,10
0,-0.436062,2.0,0.406872,-0.869472,-0.122182,-0.088238,1.0,1.0,0.0,0.0,1.0
1,0.307686,1.0,-1.070936,0.737659,0.072610,-0.035180,3.0,1.0,0.0,0.0,1.0
2,-0.247997,1.0,0.037420,0.000000,0.007679,-0.141297,3.0,1.0,0.0,0.0,0.0
3,-0.748152,2.0,0.530023,0.528034,0.137541,0.017879,1.0,1.0,0.0,0.0,1.0
4,-0.808543,1.0,-1.070936,-1.288723,-0.187113,-0.194356,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.092762,2.0,0.406872,0.108782,-0.057251,-0.088238,1.0,1.0,1.0,0.0,1.0
69996,1.269492,1.0,-0.824635,3.602545,0.072610,-0.035180,2.0,2.0,0.0,0.0,1.0
69997,-0.163286,2.0,2.254133,2.135165,0.332333,-0.035180,3.0,1.0,0.0,1.0,0.0
69998,1.200589,1.0,-0.208881,-0.170719,0.040145,-0.088238,1.0,2.0,0.0,0.0,0.0


In [8]:
imputed_X.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
dtype: int64

In [21]:
# Thay thế giá trị NaN bằng giá trị phổ biến nhất trong cột
from sklearn.impute import SimpleImputer

# Imputation, sẽ mất tên cột trong dataframe
my_impute = SimpleImputer(strategy='most_frequent')
imputed_X = pd.DataFrame(my_impute.fit_transform(X))

# Imputation removed column names; put them back
imputed_X.columns = X.columns

imputed_X = np.array(imputed_X)
# processing 
imputed_X = processing(0, imputed_X)
imputed_X = processing(2, imputed_X)
imputed_X = processing(3, imputed_X)
imputed_X = processing(4, imputed_X)
imputed_X = processing(5, imputed_X)

imputed_X = pd.DataFrame(imputed_X)

# Lưu lại dữ liệu sau khi điền đầy missing bằng giá trị phổ biến nhất
import pickle
with open('cardio/X_most_frequent.pkl', 'wb') as f:
    pickle.dump(imputed_X, f)

In [22]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='median')
imputed_X = pd.DataFrame(my_imputer.fit_transform(X))

# Imputation removed column names; put them back
imputed_X.columns = X.columns

imputed_X = np.array(imputed_X)
# processing 
imputed_X = processing(0, imputed_X)
imputed_X = processing(2, imputed_X)
imputed_X = processing(3, imputed_X)
imputed_X = processing(4, imputed_X)
imputed_X = processing(5, imputed_X)

imputed_X = pd.DataFrame(imputed_X)

# Lưu lại dữ liệu sau khi điền đầy missing bằng giá trị trung vị, trung gian
import pickle
with open('cardio/X_median.pkl', 'wb') as f:
    pickle.dump(imputed_X, f)

# Interpolate (Các phương pháp nội suy)

In [75]:
linear_interpolated_X = X.interpolate(method ='linear', limit_direction ='backward')
linear_interpolated_X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72.0,29.0,0,33.6,0.627,50
1,1,85,68.0,29.0,0,26.6,0.351,31
2,8,183,64.0,0.0,0,23.3,0.672,32
3,1,89,52.0,23.0,94,28.1,0.167,21
4,0,137,40.0,11.5,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76.0,48.0,180,32.9,0.171,63
764,2,122,70.0,27.0,0,36.8,0.340,27
765,5,121,72.0,23.0,112,26.2,0.245,30
766,1,126,60.0,0.0,0,30.1,0.349,47


In [76]:
poly_interpolated_X = X.interpolate(method='polynomial', order=5, limit_direction ='backward')
poly_interpolated_X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72.000000,NaN,0,33.6,0.627,50
1,1,85,119.968976,29.000000,0,26.6,0.351,31
2,8,183,64.000000,0.000000,0,23.3,0.672,32
3,1,89,25.616684,23.000000,94,28.1,0.167,21
4,0,137,40.000000,-11.996654,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76.000000,48.000000,180,32.9,0.171,63
764,2,122,70.000000,27.000000,0,36.8,0.340,27
765,5,121,72.000000,23.000000,112,26.2,0.245,30
766,1,126,60.000000,0.000000,0,30.1,0.349,47
